## **Please run the code on Google Colab as it might not run on Jupyter Notebook**

In [ ]:
pip install jupyter-dash -q

     |████████████████████████████████| 7.3 MB 9.2 MB/s 
     |████████████████████████████████| 25.3 MB 1.3 MB/s 
     |████████████████████████████████| 357 kB 43.4 MB/s 


In [3]:
from jupyter_dash import JupyterDash #library to create dashboard
from dash import html
from dash import dcc
# import dash_html_components as html
# import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.graph_objs as go #library to create graphs and plots
import pandas as pd

terr2 = pd.read_csv('https://raw.githubusercontent.com/sanatMahajan/DSlabDataset/main/modified_globalterrorismdb_0718dist.csv') #dataset

# Create dictionary of list
terr2_list = terr2[['country_txt', 'latitude', 'longitude']]
dict_of_locations = terr2_list.set_index('country_txt')[['latitude', 'longitude']].T.to_dict('dict')

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets) #creating the dashboard app

#layout of the app
app.layout = html.Div([
                html.Div([
                    html.Div([
                        html.Div([
                            html.H1('Global Terrorism Database', style={'text-align':'center'})
                        ])

                    ], className='six column', id = 'title')

                ], id = 'header', className='row', style={'margin-bottom': '10px'}),

                html.Div([
                    html.Div([
                                dcc.Graph(id = 'map_chart', config={'displayModeBar': 'hover'}) #world scatter map

                            ], className='twelve columns')

                ], className='row', style={'margin-bottom': '20px'}),

                html.Div([
                    html.Div([
                        html.P('Select Region', style={'margin-top': '20px'}),
                                    dcc.Dropdown(id = 'w_countries', #dropdown menu to select Region
                                                multi = False,
                                                searchable= True,
                                                value='South Asia', #default region 
                                                placeholder= 'Select Region',
                                                style={'margin-bottom': '30px'},
                                                options= [{'label': c, 'value': c} for c in (terr2['region_txt'].unique())]),

                        html.P('Select Country'),
                                    dcc.Dropdown(id = 'w_countries1', #dropdown menu to select Country
                                                multi = False,
                                                searchable= True,
                                                placeholder= 'Select Country',
                                                style={'margin-bottom': '30px'},
                                                options= []),

                        html.P('Select Year'),
                                    dcc.RangeSlider(id = 'select_years', #range slider to select the years
                                                    min = 1970,
                                                    max = 2017,
                                                    dots = False,
                                                    value= [2000, 2017]),
                    ], className='three columns'),

                    html.Div([
                          dcc.Graph(id = 'bar_chart', config={'displayModeBar': 'hover'}) #bar plot to display trend of deaths, injured and attacks

                    ], className='six columns'),

                    html.Div([
                          dcc.Graph(id = 'pie_chart', config={'displayModeBar': 'hover'}) #pie chart to display no. of deaths, injured and attacks

                    ], className='three columns')

                ], className='row')

            ], id = 'mainContainer', style={'display': 'flex', 'flex-direction': 'column', 'width': '85%', 'margin': '5% auto 5%'})

@app.callback(Output('map_chart', 'figure'), #defines what will be displayed on mapbox on selecting the options
              [Input('w_countries','value')],
              [Input('w_countries1','value')],
              [Input('select_years','value')])
def update_graph(w_countries, w_countries1, select_years):
    terr8 = terr2.groupby(['region_txt', 'country_txt', 'provstate', 'city', 'iyear', 'latitude', 'longitude'])[['nkill', 'nwound', 'attacktype1']].sum().reset_index()
    terr9 = terr8[(terr8['region_txt'] == w_countries) &
                 (terr8['country_txt'] == w_countries1) &
                 (terr8['iyear'] >= select_years[0]) & (terr8['iyear'] <= select_years[1])]

    if w_countries1:
        zoom=3
        zoom_lat = dict_of_locations[w_countries1]['latitude']
        zoom_long = dict_of_locations[w_countries1]['longitude']

    return {
        'data': [go.Scattermapbox(
            lon=terr9['longitude'],
            lat=terr9['latitude'],
            mode='markers',
            marker=go.scattermapbox.Marker(size=terr9['nwound'], #circle shown in map
                                           color=terr9['nwound'],
                                           colorscale='HSV',
                                           showscale=False,
                                           sizemode='area',
                                           opacity=0.5),
            hoverinfo='text',
            hovertext=                                                         #displays on hovering on markers
            '<b>Region</b>: ' + terr9['region_txt'].astype(str) + '<br>' + 
            '<b>Country</b>: ' + terr9['country_txt'].astype(str) + '<br>' +
            '<b>Province/State</b>: ' + terr9['provstate'].astype(str) + '<br>' +
            '<b>City</b>: ' + terr9['city'].astype(str) + '<br>' +
            '<b>Year</b>: ' + terr9['iyear'].astype(str) + '<br>' +
            '<b>Death</b>: ' + [f'{x:,.0f}' for x in terr9['nkill']] + '<br>' +
            '<b>Injured</b>: ' + [f'{x:,.0f}' for x in terr9['nwound']] + '<br>' +
            '<b>Attack</b>: ' + [f'{x:,.0f}' for x in terr9['attacktype1']] + '<br>'
        )],

        'layout': go.Layout(
            hovermode='x',
            margin=dict(r=0, l =0, b = 0, t = 0),
            mapbox=dict(
                accesstoken='pk.eyJ1Ijoic21haGFqYW4xMjMiLCJhIjoiY2t3cDgyazkwMGE3MTJ2bGN4Y2VvNmc4dyJ9.HpLQOl_l5suShRMajIZY4w',
                center = go.layout.mapbox.Center(lat=zoom_lat, lon=zoom_long),
                style='satellite-streets', #The built-in Mapbox styles are: basic, streets, outdoors, light, dark, satellite, satellite-streets 
                zoom=zoom,
            ),
            autosize=True
        )
    }

@app.callback(Output('w_countries1', 'options'), #Regions dropdown callback will give all countries in that region
              [Input('w_countries','value')])
def update_country(w_countries):
    terr3 = terr2[terr2['region_txt'] == w_countries]
    return [{'label': i, 'value': i} for i in terr3['country_txt'].unique()] 

@app.callback(Output('w_countries1', 'value'), #countries dropdown callback
              [Input('w_countries1', 'options')])
def update_country(w_countries1):
    return [k['value'] for k in w_countries1][0] #default country will be the first one in the region

@app.callback(Output('bar_chart', 'figure'), #will show bar plot on selecting the options
              [Input('w_countries','value')],
              [Input('w_countries1','value')],
              [Input('select_years','value')])
def update_graph(w_countries, w_countries1, select_years):
    terr5 = terr2.groupby(['region_txt', 'country_txt', 'iyear'])[['nkill', 'nwound', 'attacktype1']].sum().reset_index()
    terr6 = terr5[(terr5['region_txt'] == w_countries) &
                 (terr5['country_txt'] == w_countries1) &
                 (terr5['iyear'] >= select_years[0]) & (terr5['iyear'] <= select_years[1])]


    return {
        'data': [go.Scatter(           #line plot showing trends of deaths over years
            x=terr6['iyear'],
            y=terr6['nkill'],
            mode = 'markers+lines', 
            name='Death',
            line=dict(shape = 'spline', smoothing = 1.3, width = 3, color = '#FF00FF'),
            marker=dict(color='white', size = 10, symbol = 'circle',
                        line=dict(color = '#FF00FF', width = 2)),
            hoverinfo='text',
            hovertext=
            '<b>Region</b>: ' + terr6['region_txt'].astype(str) + '<br>' +
            '<b>Country</b>: ' + terr6['country_txt'].astype(str) + '<br>' +
            '<b>Year</b>: ' + terr6['iyear'].astype(str) + '<br>' +
            '<b>Death</b>: ' + [f'{x:,.0f}' for x in terr6['nkill']] + '<br>'


        ),
            go.Bar(                     #bar plot displaying no of injured people
                x=terr6['iyear'], 
                y=terr6['nwound'],
                text = terr6['nwound'],
                texttemplate='%{text:,.0f}',
                textposition='auto',
                name='injured',
                marker=dict(color='#9C0C38'),
                hoverinfo='text',
                hovertext=
                '<b>Region</b>: ' + terr6['region_txt'].astype(str) + '<br>' +
                '<b>Country</b>: ' + terr6['country_txt'].astype(str) + '<br>' +
                '<b>Year</b>: ' + terr6['iyear'].astype(str) + '<br>' +
                '<b>Injured</b>: ' + [f'{x:,.0f}' for x in terr6['nwound']] + '<br>'

            ),
            go.Bar(                     #bar chart displaying terrorist attacks 
                x=terr6['iyear'],
                y=terr6['attacktype1'],
                text=terr6['attacktype1'],
                texttemplate='%{text:,.0f}',
                textposition='auto',
                name='Attack',
                marker=dict(color='orange'),
                hoverinfo='text',
                hovertext=
                '<b>Region</b>: ' + terr6['region_txt'].astype(str) + '<br>' +
                '<b>Country</b>: ' + terr6['country_txt'].astype(str) + '<br>' +
                '<b>Year</b>: ' + terr6['iyear'].astype(str) + '<br>' +
                '<b>Attack</b>: ' + [f'{x:,.0f}' for x in terr6['attacktype1']] + '<br>'

            )
        ],

        'layout': go.Layout(            #layout of the bar plot, specifies it to be stack bar chart
            barmode = 'stack',
            title={'text': 'Death, Injured and Attack: ' + (w_countries1) + ' ' + '<br>'
                   + ' - ' .join([str(y) for y in select_years]) + '<br>',
                   'y': 0.93,
                   'x': 0.57,
                   'xanchor': 'center',
                   'yanchor': 'top'},
            titlefont={'size': 20},
            font=dict(family='sans-serif',
                      size=12),
            hovermode='closest',
            legend={'orientation': 'h',
                    'xanchor': 'center', 'x': 0.5, 'y': -0.3},
            margin=dict(r=0),
            xaxis=dict(title='<b>Year</b>',
                       tick0 = 0,
                       dtick = 1,
                       showline=True,
                       showgrid=True,
                       showticklabels=True,
                       linewidth=1,
                       ticks='outside',
                       tickfont=dict(
                           family='Aerial',
                           size=12
                       )),
            yaxis=dict(title='<b>Death, Injured and Attack</b>',
                       showline=True,
                       showgrid=True,
                       showticklabels=True,
                       linewidth=1,
                       ticks='outside',
                       tickfont=dict(
                           family='Aerial',
                           size=12
                       )
                       )
        )
    }

@app.callback(Output('pie_chart', 'figure'),   #displays the pie chart displaying no. of deaths,injured and attacks of the country
              [Input('w_countries','value')],
              [Input('w_countries1','value')],
              [Input('select_years','value')])
def update_graph(w_countries, w_countries1, select_years):
    terr7 = terr2.groupby(['region_txt', 'country_txt', 'iyear'])[['nkill', 'nwound', 'attacktype1']].sum().reset_index()
    death = terr7[(terr7['region_txt'] == w_countries) &
                  (terr7['country_txt'] == w_countries1) &
                  (terr7['iyear'] >= select_years[0]) & (terr7['iyear'] <= select_years[1])]['nkill'].sum()
    injured = terr7[(terr7['region_txt'] == w_countries) &
                  (terr7['country_txt'] == w_countries1) &
                  (terr7['iyear'] >= select_years[0]) & (terr7['iyear'] <= select_years[1])]['nwound'].sum()
    attack = terr7[(terr7['region_txt'] == w_countries) &
                    (terr7['country_txt'] == w_countries1) &
                    (terr7['iyear'] >= select_years[0]) & (terr7['iyear'] <= select_years[1])]['attacktype1'].sum()
    colors = ['#FF00FF', '#9C0C38', 'orange']

    return {
        'data': [go.Pie(          #plotly function for pie chart
            labels=['Total death', 'Total injured', 'Total attack'],
            values=[death, injured, attack],
            marker=dict(colors=colors),
            hoverinfo='label+value+percent',
            textinfo='label+value',
            rotation=45
        )],

        'layout': go.Layout(        #layout/design of the pie chart
            title={'text': 'Death, Injured and Attack: ' + '<br>' + (w_countries1) + '<br>' + ' ' +
                   ' - ' .join([str(y) for y in select_years]) ,
                   'y': 0.93,
                   'x': 0.5,
                   'xanchor': 'center',
                   'yanchor': 'top'},
            titlefont={'size': 17},
            font=dict(family='sans-serif',
                      size=12),
            hovermode='closest',
            legend={'orientation': 'h',
                    'xanchor': 'center', 'x': 0.5, 'y': -0.3}
        )
    }

# If any problem occurs like if any graph doesnot appear properly try changing the port in the below line to 8030, 8050, 8000 or any other valid port no.
app.run_server(mode='external', port=8040)

# app.run_server(mode='inline', port=8040) # To run in outout window

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning:

DataFrame columns are not unique, some columns will be omitted.



Dash app running on:


<IPython.core.display.Javascript object>